In [1]:
from ray.tune.schedulers import PopulationBasedTrainingReplay

In [8]:
policy = '4752a_00000'
replay = PopulationBasedTrainingReplay(
    f"./data/FashionMNIST-pbt/pbt_policy_{policy}.txt")

replay.config  # Initial config
  # Schedule, in the form of tuples (step, config)


{'progress_bar_refresh_rate': 0,
 'layer_1_size': 512,
 'layer_2_size': 512,
 'lr': 5e-05,
 'batch_size': 1024,
 'data_dir': '~/mldata',
 'data_mean': 0.28604063391685486,
 'data_std': 0.35302430391311646,
 'augmentations': [['blur', 0.553326343582685],
  ['rotate_left', 0.4339827645455905],
  ['rotate_right', 0.0682481991436582]]}

In [9]:
replay._policy

[(14,
  {'progress_bar_refresh_rate': 0,
   'layer_1_size': 512,
   'layer_2_size': 512,
   'lr': 5e-05,
   'batch_size': 1024,
   'data_dir': '~/mldata',
   'data_mean': 0.28604063391685486,
   'data_std': 0.35302430391311646,
   'augmentations': [['blur', 0.7218209096741113],
    ['rotate_left', 0.0728995372900052],
    ['rotate_right', 0.7306509492176126]]})]

In [10]:
from FashionMNISTLightningDataModule import FashionMNISTLightningDataModule
from augmentation.augmentation import TRANSFORM_NAMES
from FashionMNISTLightningModule import FashionMNISTLightningModule
import pytorch_lightning as pl
import os
import torch

batch_and_lr_multiplier = 5
conf = {
    "progress_bar_refresh_rate": 25,
    "layer_1_size": 512,
    "layer_2_size": 512,
    "lr": 0.00001 * batch_and_lr_multiplier,
    "batch_size": 32 * (2 ** batch_and_lr_multiplier),
    "data_dir": "./data",
    # Fashion mnist mean and std
    "data_mean": 0.28604063391685486,
    "data_std": 0.35302430391311646,
    "augmentations": [],
}

model = FashionMNISTLightningModule.load_from_checkpoint(os.path.join("./data/FashionMNIST-pbt/train_mnist_tune_checkpoint_4752a_00000_0_2021-12-12_20-06-50/checkpoint_000014", "checkpoint"), conf=conf)
data = FashionMNISTLightningDataModule(conf=conf)

trainer = pl.Trainer(
    default_root_dir="./data",
    gpus=-1 if torch.cuda.device_count() > 0 else 0,
    max_epochs=15,
    progress_bar_refresh_rate=conf["progress_bar_refresh_rate"],
    num_sanity_val_steps=0,
    # callbacks=[early_stopping]
)


# test (pass in the model)
trainer.test(model, datamodule=data)

/Users/akaver/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:90: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=25)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/opt/homebrew/lib/python3.9/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warn

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'avg_test_acc': 0.7325215339660645,
 'avg_test_loss': 1.305090069770813,
 'test_acc': 0.7325000166893005,
 'test_acc_epoch': 0.7325000166893005,
 'test_loss': 1.305486798286438,
 'test_loss_epoch': 1.305486798286438}
--------------------------------------------------------------------------------


[{'test_loss': 1.305486798286438,
  'test_loss_epoch': 1.305486798286438,
  'test_acc': 0.7325000166893005,
  'test_acc_epoch': 0.7325000166893005,
  'avg_test_loss': 1.305090069770813,
  'avg_test_acc': 0.7325215339660645}]